# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Подготовка

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Избавимся от лишних символов.

In [4]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text

In [5]:
data['text'] = data['text'].apply(clear_text)

In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,explanation why the edits made under my userna...,0
1,1,d aww he matches this background colour i m s...,0
2,2,hey man i m really not trying to edit war it...,0
3,3,more i can t make any real suggestions on im...,0
4,4,you sir are my hero any chance you remember...,0


In [7]:
#data['text'] = data['text'].astype('U')

Проведем лемматизацию

In [8]:
lemmatizer = WordNetLemmatizer()
#def lemmatize(text):
    #return ''.join(m.lemmatize(text))
def penn2morphy(penntag):
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize(text): 
    return [lemmatizer.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]
def join_list(array):
    return ' '.join(array)

In [9]:
tqdm.pandas()

data['text'] = data['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [10]:
data['text'] = data['text'].apply(join_list)

In [11]:
data['text']

0         explanation why the edits make under my userna...
1         d aww he match this background colour i m seem...
2         hey man i m really not try to edit war it s ju...
3         more i can t make any real suggestion on impro...
4         you sir be my hero any chance you remember wha...
                                ...                        
159287    and for the second time of ask when your view ...
159288    you should be ashamed of yourself that be a ho...
159289    spitzer umm theres no actual article for prost...
159290    and it look like it be actually you who put on...
159291    and i really don t think you understand i come...
Name: text, Length: 159292, dtype: object

## Обучение

Подготовим стоп слова.

In [12]:
stopwords = set(nltk_stopwords.words('english'))

Разделим данные на выборки

In [13]:
features = data['text']
target = data['toxic']

In [14]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.25)

In [15]:
features_train.shape

(119469,)

In [16]:
len(target_train)

119469

In [17]:
features_test.shape

(39823,)

In [18]:
len(target_test)

39823

Векторизация

In [19]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [20]:
tfidf_train = count_tf_idf.fit_transform(features_train)
tfidf_test = count_tf_idf.transform(features_test)

Обучим логистическую регрессию.

In [21]:
model_lin = LogisticRegression(solver='liblinear', random_state=12345, max_iter=5000, penalty='l1')
m_l = GridSearchCV(estimator=model_lin, param_grid={}, scoring='f1', cv=3, verbose=3)
m_l.fit(tfidf_train, target_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ................................................ total time=   0.8s
[CV 2/3] END ................................................ total time=   0.8s
[CV 3/3] END ................................................ total time=   0.8s


GridSearchCV(cv=3,
             estimator=LogisticRegression(max_iter=5000, penalty='l1',
                                          random_state=12345,
                                          solver='liblinear'),
             param_grid={}, scoring='f1', verbose=3)

In [22]:
m_l.best_estimator_

LogisticRegression(max_iter=5000, penalty='l1', random_state=12345,
                   solver='liblinear')

In [23]:
m_l.best_score_

0.7539468391331615

Получили нужный нам f1_score. Посмотрим, что с другими моделями.

Теперь обучим случайный лес.

In [24]:
model_forest = RandomForestClassifier(random_state=12345)
params = {'max_depth': range(5,10), 'n_estimators': range(25, 51, 25)}
m_f = GridSearchCV(estimator=model_forest, param_grid=params, scoring='f1', cv=3, verbose=3)
m_f.fit(tfidf_train, target_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END ...................max_depth=5, n_estimators=25; total time=   2.0s
[CV 2/3] END ...................max_depth=5, n_estimators=25; total time=   1.9s
[CV 3/3] END ...................max_depth=5, n_estimators=25; total time=   1.9s
[CV 1/3] END ...................max_depth=5, n_estimators=50; total time=   3.7s
[CV 2/3] END ...................max_depth=5, n_estimators=50; total time=   3.6s
[CV 3/3] END ...................max_depth=5, n_estimators=50; total time=   3.7s
[CV 1/3] END ...................max_depth=6, n_estimators=25; total time=   2.2s
[CV 2/3] END ...................max_depth=6, n_estimators=25; total time=   2.2s
[CV 3/3] END ...................max_depth=6, n_estimators=25; total time=   2.2s
[CV 1/3] END ...................max_depth=6, n_estimators=50; total time=   4.3s
[CV 2/3] END ...................max_depth=6, n_estimators=50; total time=   4.3s
[CV 3/3] END ...................max_depth=6, n_e

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': range(5, 10),
                         'n_estimators': range(25, 51, 25)},
             scoring='f1', verbose=3)

In [25]:
m_f.best_estimator_

RandomForestClassifier(max_depth=5, n_estimators=25, random_state=12345)

In [26]:
m_f.best_score_

0.0

## Тестирование

Посчитаем f1_score модели логистической регрессии на тестовой выборке

In [27]:
f1_score(target_test, m_l.best_estimator_.predict(tfidf_test))

0.7745387973524855

Получили нужное нам значение метрики f1_score = 0.7745387973524855

## Вывод

Вывод.  
1. Данные были загружены.
2. Данные были обработаны с помощью регулярных выражений и лемматизации.
3. Данные разделены на выборки(тренировочную и тестовую)
4. Была произведена векторизациия данных.
5. Были обучены модели и найдена лучшая LogisticRegression.
6. На тестовой выборке с моделью LogisticRegression был получен нужный нам f1_score(> 0.75) 0.7745387973524855